In [1]:
import pandas as pd
import numpy as np
import math as m

In [52]:
data=pd.read_csv('DS3/data.csv')
data_T=pd.DataFrame(data, columns= ['Object','Property','Value','Source'])
data_T

Object   Property   Value    Source
0          Object1  Property4  373418   Source1
1          Object1  Property2  224546   Source1
2          Object2  Property4  484579   Source1
3          Object2  Property2  390812   Source1
4          Object3  Property4  314886   Source1
...            ...        ...     ...       ...
999995  Object1996  Property8  748696  Source48
999996  Object1997  Property8  773471  Source48
999997  Object1998  Property8  594826  Source48
999998  Object1999  Property8  702357  Source48
999999  Object2000  Property8  800929  Source48

[1000000 rows x 4 columns]

In [53]:
data_T['TSInitial']=0.8
data_T['C_v']=0.0
data_T.head(5)

Object   Property   Value   Source  TSInitial  C_v
0  Object1  Property4  373418  Source1        0.8  0.0
1  Object1  Property2  224546  Source1        0.8  0.0
2  Object2  Property4  484579  Source1        0.8  0.0
3  Object2  Property2  390812  Source1        0.8  0.0
4  Object3  Property4  314886  Source1        0.8  0.0

In [54]:
# Definition de la matice de similarite
def similarity(dataframe):
    dict_sim = {}
    for key, df in dataframe.groupby(by=['Object','Property']):
        Values = df['Value'].unique()
        row = key[0]+key[1]
        for  i in range(len(Values)):
            w1 = Values[i]
            for  u in range(len(Values)):
                w2 = Values[u]
                sim = 1
                if w1!=w2:
                    t = abs(w1-w2)
                    sim = 1/t
                dict_sim[row+str(w1)+str(w2)] = sim
                dict_sim[row+str(w2)+str(w1)] = sim
    return dict_sim

In [55]:
import math
def Ts(df):
    ts_list = df['TSInitial'].to_list()
    return sum([-math.log(1-ts) for ts in ts_list])
def cos_sim(a,b):
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

In [57]:
import time as t
from natsort import index_natsorted, order_by_index
def TruthFinder_seq(data_T):
    ni=5
    trustold = 0.00001
    start = t.time()
    nb_iter_do = 0
    rho=0.7
    lam=0.5
    dict_sim=similarity(data_T)
    for i in range(ni):
        df = data_T[['Source', 'TSInitial']].drop_duplicates().reindex(index=order_by_index(data_T[['Source', 'TSInitial']].drop_duplicates().index, index_natsorted(data_T[['Source', 'TSInitial']].drop_duplicates()['Source'], reverse=False)))
        ts1=df['TSInitial']
        grouped_df = data_T.groupby(["Object", "Property","Value"])
        for key,item in grouped_df:
            sum_ = Ts(item)
            data_T.loc[((data_T["Object"] == key[0]) & (data_T["Property"]==key[1]) & (data_T["Value"]==key[2])), "C_v"]=sum_
        grouped_df=data_T.groupby(["Object", "Property","Value"])
        for key ,item in grouped_df:
            cv=data_T.loc[((data_T["Object"] == key[0]) & (data_T["Property"]==key[1]) & (data_T["Value"]==key[2])), "C_v"].values[0]
#             print(cv)

            data_d=data_T.loc[((data_T["Object"] == key[0]) & (data_T["Property"]==key[1]) & (data_T["Value"]!=key[2])), ["Value","C_v"]]
            for row in data_d.values:
                Key_sim=key[0]+key[1]+str(key[2])+str(int(row[0]))
                cv+=rho*(row[1]*dict_sim[Key_sim])
            cv=1/(1+np.exp(-lam*cv))
        #     print(cv)
            data_T.loc[((data_T["Object"] == key[0]) & (data_T["Property"]==key[1]) & (data_T["Value"]==key[2])), "C_v"]=cv
        groupe_ts=data_T.groupby(['Source'])
        for key,item in groupe_ts:
            ts_=item['C_v'].mean()
            data_T.loc[((data_T["Source"] == key)), "TSInitial"]=ts_
        df = data_T[['Source', 'TSInitial']].drop_duplicates().reindex(index=order_by_index(data_T[['Source', 'TSInitial']].drop_duplicates().index, index_natsorted(data_T[['Source', 'TSInitial']].drop_duplicates()['Source'], reverse=False)))
        ts2=df['TSInitial']
        print('Itération : ',i+1)
        print(1-abs(cos_sim(ts1,ts2)))
        nb_iter_do = i+1
        if 1-abs(cos_sim(ts1,ts2)) < trustold:
            break
    end = t.time()

    return data_T,end-start,nb_iter_do



In [58]:
result_=TruthFinder_seq(data_T)

Itération :  1
0.0001908140383467538
Itération :  2
8.644064012464092e-06


In [59]:
result_

(            Object   Property   Value    Source  TSInitial       C_v
 0          Object1  Property4  373418   Source1   0.979305  1.000000
 1          Object1  Property2  224546   Source1   0.979305  1.000000
 2          Object2  Property4  484579   Source1   0.979305  1.000000
 3          Object2  Property2  390812   Source1   0.979305  1.000000
 4          Object3  Property4  314886   Source1   0.979305  1.000000
 ...            ...        ...     ...       ...        ...       ...
 999995  Object1996  Property8  748696  Source48   0.949695  0.991229
 999996  Object1997  Property8  773471  Source48   0.949695  0.769564
 999997  Object1998  Property8  594826  Source48   0.949695  0.977908
 999998  Object1999  Property8  702357  Source48   0.949695  0.982952
 999999  Object2000  Property8  800929  Source48   0.949695  0.913661
 
 [1000000 rows x 6 columns],
 494049.1006760597,
 2)

In [60]:
def res(result):
    gr=result.groupby(["Object", "Property"])
    data_out={'Object':[],'Property':[],'Value':[]}
    for key,item in gr:
#         print(item.values)
        cv_max=0
        value_pred=None
        for line in item.values:
#             print(line[5])
            if cv_max<line[5]:
                cv_max=line[5]
                value_pred=line[2]
        data_out['Object'].append(key[0])
        data_out['Property'].append(key[1])
        data_out['Value'].append(value_pred)
    return pd.DataFrame(data_out)
        
        

In [65]:
res_=res(result_[0])
len(res_)

20000

In [66]:
list_ = res_.to_numpy()

In [67]:
data_truth=pd.read_csv('DS3/data_truth.csv')
data_truth_=pd.DataFrame(data_truth, columns= ['Object','Property','Value'])


In [68]:
len(data_truth_)

20000

In [69]:
list__=data_T.to_numpy()
# list__

In [70]:
list_1=data_truth_.to_numpy()


In [71]:
# def function(list_A,list_B):
#     binary=[]
#     search_list = [tuple(li[:-3]) for li in list_B]
#     for ser_item in search_list:
#         if ser_item in list_A:
#             binary.append(1)
#         else:
#             binary.append(0)
#     return binary

In [72]:
def function(list_A,list_B):
    binary=[]
    search_list = [tuple(li[:-3]) for li in list_B]
    list_A = [tuple(li) for li in list_A]
    for ser_item in search_list:
        if ser_item in list_A:
            binary.append(1)
        else:
            binary.append(0)
    return binary

In [73]:
res = function(list_, list__)
res_truth=function(list_1, list__)
# res
# res_truth

In [45]:
# !pip install scikit-learn
import sklearn

In [46]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [48]:
precision_score(res_truth,res)

0.8861625008575544

In [49]:
accuracy_score(res_truth,res)

0.9216

In [50]:
recall_score(res_truth,res)

0.9312234157594982

In [51]:
f1_score(res_truth,res)

0.9081343301070984